In [1]:
from datasets import load_dataset, concatenate_datasets
import json
import torch
from datasets.utils.logging import disable_progress_bar,enable_progress_bar
import os
import numpy as np


In [2]:
dataset_name = "multirc_n_shot"
dataset_name_hf = "hassansh/"+ dataset_name # The dataset name in Hugging Face Datasets

num_shots = 0
dataset = load_dataset(dataset_name_hf, name = f"{num_shots}_shot")['test']
idx = range(dataset.num_rows)
dataset = dataset.add_column("idx", idx)
print(dataset)

# make a dict of questions for the same paragraph
disable_progress_bar()
num_qs_all = 953
# q_list = torch.zeros(dataset.num_rows)
q_list = {}
ans_list = {}
# ans_list = torch.zeros(dataset.num_rows)
step = 100
for q in range(num_qs_all):
    dd = dataset.filter(lambda example: example['question']==q)
    q_list[f"{q}"] = []
    ans_list[f"{q}"] = []
    for ind in range(dd.num_rows):
        # q_list[dd[ind]['idx']] = q
        # ans_list[dd[ind]['idx']] = dd[ind]['target']
        q_list[f"{q}"].append(dd[ind]['idx'])
        ans_list[f"{q}"].append(dd[ind]['target'])

    if q % step == 0:
        print(f"{q//step}", end='\r')
# print(num_qs_all)
enable_progress_bar()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'input', 'target_str', 'target', 'idx'],
    num_rows: 4848
})


In [120]:
print(ans_list["1"])

[0, 0, 1, 1, 0]


In [19]:
model_name = "Llama-2-7b-hf/"
# model_name = "Mistral-7B-v0.1/"
folder_in = "gromovand/" + model_name
num_shots = 0
path_to_results = folder_in + dataset_name + f"_{num_shots}/"
log_address_acc = os.path.join(path_to_results, "accuracy_per_question.json")

with open(log_address_acc, 'r') as log_file:
    # Read the log file line by line
    for line in log_file:
        acc_list = json.loads(line)

acc_list = torch.tensor(acc_list, dtype= torch.int8)
print(acc_list.shape)

torch.Size([4848])


In [20]:
F1_list = []
noTpos =[]
nopos =[]
EM = 0 # exact match
for q in range(num_qs_all):
    # print(acc_list[q_list[f"{q}"]])
    TA_arr = torch.argwhere(acc_list[q_list[f"{q}"]]==1)[:,0] # correc answers
    FA_arr = torch.argwhere(acc_list[q_list[f"{q}"]]==0)[:,0] # wrong answers
    GP_arr = torch.argwhere(torch.tensor(ans_list[f"{q}"])==1)[:,0] # ground positive
    GN_arr = torch.argwhere(torch.tensor(ans_list[f"{q}"])==0)[:,0] # ground negative
    # print(q, TA,FA,GP,GN)
    GP = GP_arr.shape[0]
    TP = np.intersect1d(TA_arr,GP_arr).shape[0]
    TN = np.intersect1d(TA_arr,GN_arr).shape[0]
    FP = np.intersect1d(FA_arr,GN_arr).shape[0]
    FN = np.intersect1d(FA_arr,GP_arr).shape[0]
    if FA_arr.shape[0]==0:
        # print(q)
        EM += 1
    # print(TP.shape, TN.shape)
    # if (TP.shape[0]+ FP.shape[0])> 0:
        # if TP.shape[0]>0:
            # precision = TP.shape[0]/(TP.shape[0]+ FP.shape[0])
            # recall = TP.shape[0]/GP.shape[0]
            # F1_list.append( 2*precision*recall/(precision+recall))
    if TP+FP+GP >0 :
        F1_list.append(2*TP/(TP+FP+ GP))
    # else:
    #     F1_list.append(1)

    #     # else:
    #         noTpos.append(q)
    # else:
    #     nopos.append(q)

F1_list = torch.tensor(F1_list)
print(f"F1_a: {F1_list.mean().item():.3f}, {F1_list.std().item()/(F1_list.shape[0]-1)**0.5:.3f}, over {F1_list.shape[0]}")
print(f"EM: {EM/num_qs_all:.3f}")

# # for i in range(acc_list.shape[0]):
TA = torch.argwhere(acc_list==1) # correc answers
FA = torch.argwhere(acc_list==0) # wrong answers
ans_full_list = dataset['target']
GP = torch.argwhere(torch.tensor(ans_full_list)==1) # ground positive
GN = torch.argwhere(torch.tensor(ans_full_list)==0) # ground negative
TP = np.intersect1d(TA,GP)
TN = np.intersect1d(TA,GN)
FP = np.intersect1d(FA,GN)
FN = np.intersect1d(FA,GP)
# print(TP.shape, TN.shape)
precision = TP.shape[0]/(TP.shape[0]+ FP.shape[0])
recall = TP.shape[0]/GP.shape[0]
F1 = 2*precision*recall/(precision+recall)

print(f"F1_tot: {F1:.3f}")


F1_a: 0.550, 0.008, over 952
EM: 0.042
F1_tot: 0.593


Llama-2-7B

w prompt
F1_a: 0.327, 0.011, over 952
EM: 0.049
F1_tot: 0.437

w/o prompt True/False:
F1_a: 0.197, 0.010, over 952
EM: 0.024
F1_tot: 0.336

F1_a: 0.550, 0.008, over 952
EM: 0.042
F1_tot: 0.593

---------------------
Mistral:
F1_a: 0.626, 0.007, over 953
EM: 0.083
F1_tot: 0.638

F1_a: 0.728, 0.008, over 953
EM: 0.275
F1_tot: 0.741

# 0.4721526503562927 0.011244376819454057 953
# 0.5733551383018494 0.0068698085200768555 953
# torch.argwhere(torch.tensor(ans_full_list)==1)

In [11]:
print(len(nopos), len(noTpos))
idx = 10
arr = noTpos
print(arr[idx])
print(acc_list[q_list[f"{arr[idx]}"]].tolist())
print( ((acc_list[q_list[f"{arr[idx]}"]] + 2+~torch.tensor(ans_list[f"{arr[idx]}"])) %2).tolist())
print(ans_list[f"{arr[idx]}"])


0 0


IndexError: list index out of range

In [93]:
2+~torch.tensor([0,1])

tensor([1, 0])